<a href="https://colab.research.google.com/github/yogendra1109/Machine-learning/blob/main/email_click_through_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.0 MB

In [2]:
pip install llvmlite -q

In [3]:
pip install --upgrade pycaret -q

In [4]:
from pycaret.classification import setup, compare_models

In [5]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.5 MB/s eta 0:00:00


In [6]:
pip install lightgbm

In [7]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# all the required models
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# for model evaluation
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
train_path = '/content/drive/My Drive/Hackathon/train3.csv'
test_path = '/content/drive/My Drive/Hackathon/test3.csv'
sample_path = '/content/drive/My Drive/Hackathon/sample3.csv'

In [10]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sample_df = pd.read_csv(sample_path)

In [11]:
train_df.head()

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,no_of_CTA,mean_CTA_len,is_image,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,click_rate
0,1,3,76,10439,39,5,1,Noon,6,26,3,29,0,0,0,0,0,0,0,0,14,0.103079
1,2,3,54,2570,256,5,1,Morning,2,11,0,22,0,0,0,0,0,0,0,0,10,0.700000
2,3,3,59,12801,16,5,1,Noon,2,11,3,23,1,0,1,0,0,0,0,0,16,0.002769
3,4,3,74,11037,30,4,0,Evening,15,9,4,24,0,0,0,0,0,0,0,0,10,0.010868
4,5,3,80,10011,27,5,1,Noon,6,26,3,31,0,0,1,0,0,0,0,0,14,0.142826


In [12]:
test_df.head()

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,no_of_CTA,mean_CTA_len,is_image,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience
0,1889,3,61,12871,11,6,1,Noon,6,26,3,37,1,0,0,0,1,0,0,0,14
1,1890,3,54,2569,256,5,1,Morning,2,11,0,22,0,0,0,0,0,0,0,0,10
2,1891,3,88,1473,78,4,0,Evening,2,11,0,22,1,0,1,0,0,0,0,0,10
3,1892,3,88,1473,78,3,0,Evening,2,11,0,22,1,0,1,0,0,0,0,0,10
4,1893,3,78,9020,29,3,0,Noon,6,26,2,40,0,0,0,0,0,0,0,0,14


In [13]:
sample_df.head()

,campaign_id,click_rate
0,1889,0.043454
1,1890,0.043454
2,1891,0.043454
3,1892,0.043454
4,1893,0.043454


In [14]:
target = train_df['click_rate'] # store click_rate from train set in new varible
camp_id = test_df['campaign_id'] # store campaign_id from test set in new variable

# dropping redudant columns from both sets and target column from train set, so the number of columns is same for both sets
train_df1 = train_df.drop(['campaign_id', 'click_rate','is_timer'], axis=1)
test_df1 = test_df.drop(['campaign_id','is_timer'], axis =1)

df = pd.concat([train_df1, test_df1]).reset_index(drop=True)

In [15]:
df.head()

,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,no_of_CTA,mean_CTA_len,is_image,is_personalised,is_quote,is_emoticons,is_discount,is_price,is_urgency,target_audience
0,3,76,10439,39,5,1,Noon,6,26,3,29,0,0,0,0,0,0,0,14
1,3,54,2570,256,5,1,Morning,2,11,0,22,0,0,0,0,0,0,0,10
2,3,59,12801,16,5,1,Noon,2,11,3,23,1,0,1,0,0,0,0,16
3,3,74,11037,30,4,0,Evening,15,9,4,24,0,0,0,0,0,0,0,10
4,3,80,10011,27,5,1,Noon,6,26,3,31,0,0,1,0,0,0,0,14


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2650 entries, 0 to 2649
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sender              2650 non-null   int64 
 1   subject_len         2650 non-null   int64 
 2   body_len            2650 non-null   int64 
 3   mean_paragraph_len  2650 non-null   int64 
 4   day_of_week         2650 non-null   int64 
 5   is_weekend          2650 non-null   int64 
 6   times_of_day        2650 non-null   object
 7   category            2650 non-null   int64 
 8   product             2650 non-null   int64 
 9   no_of_CTA           2650 non-null   int64 
 10  mean_CTA_len        2650 non-null   int64 
 11  is_image            2650 non-null   int64 
 12  is_personalised     2650 non-null   int64 
 13  is_quote            2650 non-null   int64 
 14  is_emoticons        2650 non-null   int64 
 15  is_discount         2650 non-null   int64 
 16  is_price            2650

In [17]:
df.isnull().sum()

sender                0
subject_len           0
body_len              0
mean_paragraph_len    0
day_of_week           0
is_weekend            0
times_of_day          0
category              0
product               0
no_of_CTA             0
mean_CTA_len          0
is_image              0
is_personalised       0
is_quote              0
is_emoticons          0
is_discount           0
is_price              0
is_urgency            0
target_audience       0
dtype: int64

In [18]:
df.duplicated().sum()

0

In [19]:
df.describe()

,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,category,product,no_of_CTA,mean_CTA_len,is_image,is_personalised,is_quote,is_emoticons,is_discount,is_price,is_urgency,target_audience
count,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000,2650.000000
mean,4.353585,86.321887,14185.344151,34.648679,2.834717,0.202642,9.846415,17.625283,4.219623,30.210566,0.919245,0.052830,0.843019,0.232453,0.037358,34.675472,0.112830,11.638491
std,3.236710,30.265757,7283.856366,27.796352,1.771125,0.402043,5.306196,12.203647,4.433314,11.596079,0.869499,0.223737,1.032265,0.735963,0.189674,497.222896,0.316445,2.973373
min,0.000000,9.000000,23.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,68.000000,9596.000000,21.000000,1.000000,0.000000,6.000000,9.000000,2.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000
50%,3.000000,83.000000,12699.000000,28.000000,3.000000,0.000000,10.000000,12.000000,3.000000,29.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,12.000000
75%,3.000000,98.000000,17279.000000,40.000000,4.000000,0.000000,15.000000,26.000000,5.000000,35.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,12.000000
max,15.000000,265.000000,44491.000000,498.000000,6.000000,1.000000,15.000000,44.000000,49.000000,162.000000,6.000000,1.000000,6.000000,17.000000,1.000000,14999.000000,1.000000,16.000000


In [20]:
df['is_price'].value_counts()

0        2633
5499        5
5000        4
8000        2
4999        2
1399        1
1000        1
14999       1
999         1
Name: is_price, dtype: int64

In [21]:
df.drop('is_price', axis=1, inplace=True)

In [22]:
df.rename(columns={'is_image':'no_image','is_quote':'no_quote','is_emoticons':'no_emoticons'}, inplace=True)

In [23]:
df1 = df.copy()

In [24]:
cat_cols = ['sender','day_of_week','category','product','target_audience']

In [25]:
sender_map = df1['sender'].value_counts().to_dict()
df1['sender'] = df1['sender'].map(sender_map)

In [26]:
day_of_week_map = df1['day_of_week'].value_counts().to_dict()
df1['day_of_week'] = df1['day_of_week'].map(day_of_week_map)

In [27]:
category_map = df1['category'].value_counts().to_dict()
df1['category'] = df1['category'].map(category_map)

In [28]:
product_map = df1['product'].value_counts().to_dict()
df1['product'] = df1['product'].map(product_map)

In [29]:
audience_map = df1['target_audience'].value_counts().to_dict()
df1['target_audience'] = df1['target_audience'].map(audience_map)

In [30]:
df2 = pd.get_dummies(df1, drop_first=True)
df2.head()

,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,category,product,no_of_CTA,mean_CTA_len,no_image,is_personalised,no_quote,no_emoticons,is_discount,is_urgency,target_audience,times_of_day_Morning,times_of_day_Noon
0,2201,76,10439,39,336,1,342,99,3,29,0,0,0,0,0,0,245,0,1
1,2201,54,2570,256,336,1,244,244,0,22,0,0,0,0,0,0,198,1,0
2,2201,59,12801,16,336,1,244,244,3,23,1,0,1,0,0,0,191,0,1
3,2201,74,11037,30,464,0,1146,467,4,24,0,0,0,0,0,0,198,0,0
4,2201,80,10011,27,336,1,342,99,3,31,0,0,1,0,0,0,245,0,1


In [31]:
train_final = df2.loc[:train_df.index.max(), :].copy()
test_final = df2.loc[train_df.index.max() + 1:, :].reset_index(drop=True).copy()

In [32]:
train_final.head()

,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,category,product,no_of_CTA,mean_CTA_len,no_image,is_personalised,no_quote,no_emoticons,is_discount,is_urgency,target_audience,times_of_day_Morning,times_of_day_Noon
0,2201,76,10439,39,336,1,342,99,3,29,0,0,0,0,0,0,245,0,1
1,2201,54,2570,256,336,1,244,244,0,22,0,0,0,0,0,0,198,1,0
2,2201,59,12801,16,336,1,244,244,3,23,1,0,1,0,0,0,191,0,1
3,2201,74,11037,30,464,0,1146,467,4,24,0,0,0,0,0,0,198,0,0
4,2201,80,10011,27,336,1,342,99,3,31,0,0,1,0,0,0,245,0,1


In [33]:
test_final.head()

,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,category,product,no_of_CTA,mean_CTA_len,no_image,is_personalised,no_quote,no_emoticons,is_discount,is_urgency,target_audience,times_of_day_Morning,times_of_day_Noon
0,2201,61,12871,11,201,1,342,99,3,37,1,0,0,1,0,0,245,0,1
1,2201,54,2569,256,336,1,244,244,0,22,0,0,0,0,0,0,198,1,0
2,2201,88,1473,78,464,0,244,244,0,22,1,0,1,0,0,0,198,0,0
3,2201,88,1473,78,472,0,244,244,0,22,1,0,1,0,0,0,198,0,0
4,2201,78,9020,29,472,0,342,99,2,40,0,0,0,0,0,0,245,0,1


In [34]:
target.head()

0    0.103079
1    0.700000
2    0.002769
3    0.010868
4    0.142826
Name: click_rate, dtype: float64

In [35]:
data=pd.concat([train_final, target], axis=1)

In [37]:
clf_setup = setup(data, target='click_rate', data_split_stratify=True)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [38]:
print(data['click_rate'].value_counts())

0.000000    39
0.500000     5
0.083333     4
0.033333     3
0.008403     3
            ..
0.019727     1
0.002228     1
0.013577     1
0.011013     1
0.003644     1
Name: click_rate, Length: 1810, dtype: int64


In [39]:
data['binary_click'] = (data['click_rate'] > 0.1).astype(int)

In [41]:
clf_setup = setup(data, target='binary_click', data_split_stratify=True)

,Description,Value
0,Session id,3186
1,Target,binary_click
2,Target type,Binary
3,Original data shape,"(1888, 21)"
4,Transformed data shape,"(1888, 21)"
5,Transformed train set shape,"(1321, 21)"
6,Transformed test set shape,"(567, 21)"
7,Numeric features,20
8,Preprocess,True
9,Imputation type,simple


In [42]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.3520
dt,Decision Tree Classifier,0.9992,0.9996,1.0000,0.9938,0.9968,0.9963,0.9964,0.0290
rf,Random Forest Classifier,0.9992,1.0000,0.9938,1.0000,0.9968,0.9963,0.9964,0.1940
ada,Ada Boost Classifier,0.9992,0.9996,1.0000,0.9938,0.9968,0.9963,0.9964,0.0310
gbc,Gradient Boosting Classifier,0.9992,0.9996,1.0000,0.9938,0.9968,0.9963,0.9964,0.1320
xgboost,Extreme Gradient Boosting,0.9985,1.0000,0.9875,1.0000,0.9935,0.9927,0.9928,0.0640
catboost,CatBoost Classifier,0.9985,1.0000,1.0000,0.9879,0.9937,0.9929,0.9930,2.3070
et,Extra Trees Classifier,0.9781,0.9974,0.8204,0.9929,0.8965,0.8844,0.8906,0.1670
lda,Linear Discriminant Analysis,0.9667,0.9993,0.7133,1.0000,0.8231,0.8060,0.8249,0.0300
ridge,Ridge Classifier,0.9508,0.0000,0.5779,1.0000,0.7222,0.6984,0.7350,0.0250


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=3186, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [43]:
models = {
    'catboost':CatBoostRegressor(verbose=0,random_state=43),
    'xgb':XGBRegressor(random_state=43),
    'lgbm':LGBMRegressor(random_state=43),
    'rf':RandomForestRegressor(random_state=43),
    'gbr':GradientBoostingRegressor(random_state=43),
    'extr':ExtraTreesRegressor(random_state=43)
}

In [44]:
for name, model in models.items():
    model.fit(train_final, target)
    print(name + ' Trained')

catboost Trained
xgb Trained
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000309 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 670
[LightGBM] [Info] Number of data points in the train set: 1888, number of used features: 19
[LightGBM] [Info] Start training from score 0.041888
lgbm Trained
rf Trained
gbr Trained
extr Trained


In [45]:
for name, model in models.items():
    print('------------\n' + name)
    print(models[name].score(train_final, target))

------------
catboost
0.9406438506898985
------------
xgb
0.9915168358847216
------------
lgbm
0.8484249549985619
------------
rf
0.9374007230722169
------------
gbr
0.7169323615420721
------------
extr
0.9999994548852689


In [46]:
mean_prediction = (
    0.2 * (models['catboost'].score(train_final, target)) +
    0.2 * (models['xgb'].score(train_final, target)) +
    0.1 * (models['lgbm'].score(train_final, target)) +
    0.2 * (models['rf'].score(train_final, target)) +
    0.1 * (models['gbr'].score(train_final, target)) +
    0.2 * (models['extr'].score(train_final, target))
    )

In [47]:
mean_prediction

0.9304479045604845

In [48]:
results = {}
for name, model in models.items():
    result = cross_val_score(model, train_final, target, cv=KFold(n_splits=10))
    results[name] = result

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000289 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 661
[LightGBM] [Info] Number of data points in the train set: 1699, number of used features: 19
[LightGBM] [Info] Start training from score 0.042058
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000239 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 661
[LightGBM] [Info] Number of data points in the train set: 1699, number of used features: 19
[LightGBM] [Info] Start training from score 0.041464
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000269 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, y

In [49]:
for name, result in results.items():
    print('------------\n' + name)
    print(np.mean(result))
    print(np.std(result))

------------
catboost
0.04279055057341021
0.3220381313338308
------------
xgb
-0.22663346986194735
0.6261792267631113
------------
lgbm
0.07285704144891154
0.30529905158696685
------------
rf
0.053628906585509804
0.3115612014605788
------------
gbr
0.07816023756337546
0.32882410033822373
------------
extr
-0.10559256734310526
0.5733515717012863


In [50]:
final_prediction = (
    0.2 * (models['catboost'].predict(test_final)) +
    0.2 * (models['xgb'].predict(test_final)) +
    0.2 * (models['lgbm'].predict(test_final)) +
    0.2 * (models['rf'].predict(test_final)) +
    0.2 * (models['extr'].predict(test_final))
    )

In [51]:
submission = pd.concat([camp_id, pd.Series(final_prediction, name='click_rate')], axis=1)

In [52]:
submission.head()

,campaign_id,click_rate
0,1889,0.067258
1,1890,0.618710
2,1891,0.211440
3,1892,0.244290
4,1893,0.143112


In [53]:
submission.to_csv('./submission.csv', index=False, header=True)